In [18]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


c:\Users\didia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [19]:
'''
df = pd.read_csv('factnews_dataset.csv') 

#criar grupo treino e teste
train_df, test_df = train_test_split(df, test_size=0.7, random_state=42)

#guardar conjuntos de treino e teste em memória
train_df.to_csv("train.csv", index=False) 
test_df.to_csv("test.csv", index=False)

'''

'\ndf = pd.read_csv(\'factnews_dataset.csv\') \n\n#criar grupo treino e teste\ntrain_df, test_df = train_test_split(df, test_size=0.7, random_state=42)\n\n#guardar conjuntos de treino e teste em memória\ntrain_df.to_csv("train.csv", index=False) \ntest_df.to_csv("test.csv", index=False)\n\n'

In [20]:
#carregar conjuntos de treino e teste
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])

#matizar
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
print(train['stems'].head())

NameError: name 'stemmer' is not defined

Análise exploratória

## TF-IDF - fred

In [ ]:
tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
max_df =0.5 , max_features =500)
p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [ ]:
c_vect = CountVectorizer(ngram_range=(1,3), strip_accents='unicode',max_features=1000)

#juntar os tokens matizados em frases novvamente 
text_train_count_vects = train['stems'].apply(lambda tokens: ' '.join(tokens))
text_test_count_vects = test['stems'].apply(lambda tokens: ' '.join(tokens))
print(text_train_count_vects.head())

count_vects_train = c_vect.fit_transform(text_train_count_vects)
count_vects_test = c_vect.transform(text_test_count_vects)

0    em mai , famíl sand homenage assemble legisl e...
1    jobim aceit convit lul assum carg 16h hoj , pa...
2    além ocup , acid vitim pesso terr , funcion em...
3                        log , assum vice-presid câm .
4                                 tem espírit públic .
Name: stems, dtype: object


In [ ]:
y_train = train['classe']
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(count_vects_train, y_train)

predicted = clf.predict(count_vects_test)

y_test = test['classe']
for doc, pred, true in zip(text_test_count_vects, predicted, y_test):
    print(f"{doc[:80]}... => Previsto: {pred} (Verdadeiro: {true})")


no prim turn , o candidat da esquerd chilen lider no brasil com 49,76 % , enquan... => Previsto: 0 (Verdadeiro: 0)
est é a segund vez em menos de um ano que o campu é fech dev a um tirotei .... => Previsto: 0 (Verdadeiro: 0)
a marg de err é doi pont par mais ou par menos .... => Previsto: 0 (Verdadeiro: 0)
pass um dia na cidad nest seman -mor no rio por motiv profissionais- e trê assal... => Previsto: 0 (Verdadeiro: -1)
na ocas , o peemedeb discord do outr doi rela e ped a absolv de renan .... => Previsto: 0 (Verdadeiro: 0)
acm era cardiopat , port de insufici card conges , em decorr de um infart , ocor... => Previsto: 0 (Verdadeiro: 0)
agor , 53 .... => Previsto: 0 (Verdadeiro: -1)
segund anal , no discurs de hoj , xi reforç o últ livr branc do part comun , que... => Previsto: 0 (Verdadeiro: 0)
de acord com o secret , câm de seguranç de estabelec próx , imag de cinegraf que... => Previsto: 0 (Verdadeiro: 0)
a reduç signific no recurs do mais médic -reb pel atual govern de médic pel br

In [ ]:
from sklearn import metrics
lista_y = [str(c) for c in (train['classe'].unique())]

print(metrics.classification_report(
y_test, predicted, target_names=set(lista_y)))

print(metrics.confusion_matrix(y_test, predicted))

              precision    recall  f1-score   support

           0       0.51      0.66      0.58       982
           1       0.81      0.81      0.81      2957
          -1       0.21      0.06      0.10       395

    accuracy                           0.71      4334
   macro avg       0.51      0.51      0.49      4334
weighted avg       0.69      0.71      0.69      4334

[[ 645  323   14]
 [ 492 2386   79]
 [ 118  252   25]]


## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-cbow-300d

In [22]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file
from gensim.models import KeyedVectors
def word2vec():
    #descarregar os embeddings do modelo Word2Vec CBOW 300d do NILC
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                           filename="embeddings.safetensors")   
    
    #carregar os vetores do ficheiro safetensors
    data = load_file(path)
    vectors = data["embeddings"]    

    #descarregar vocabulário
    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                                 filename="vocab.txt")
    
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]  

    #print(vectors.shape)
    #print(vectors)

    model = KeyedVectors(vector_size=300)
    model.add_vectors(vocab, vectors)

    print(f"Número de palavras no modelo: {len(model.key_to_index)}")
    print(f"Dimensão dos vetores: {model.vector_size}")


word2vec()


Número de palavras no modelo: 929603
Dimensão dos vetores: 300


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)